# 02 - Baseline Drift Analysis

Analyze position and orientation drift from traditional IMU integration:
- Compare pure integration vs Madgwick filter
- Measure drift at 10s, 30s, 60s horizons
- Visualize trajectory divergence from ground truth

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from src.baselines.traditional_integration import TraditionalIntegration
from src.baselines.madgwick_filter import MadgwickFilter
from src.data.kaist_imu_loader import KAISTIMUDataset
from src.data.visualization import plot_trajectory_2d, plot_drift_analysis

%matplotlib inline

In [ ]:
# Load data
dataset = KAISTIMUDataset('data/processed', ['urban01'], window_size=60.0, stride=30.0, normalize=False)
baseline = TraditionalIntegration(sampling_rate=100)

# Evaluate drift
for i in range(min(5, len(dataset))):
    sample = dataset[i]
    imu = sample['imu'].unsqueeze(0)
    result = baseline.predict(imu)
    if 'delta_position' in sample:
        error = torch.norm(result['delta_position'] - sample['delta_position'].unsqueeze(0)).item()
        print(f'Window {i}: drift = {error:.2f} m')